In [29]:
#importing libraries
import pandas as pd
import numpy as np
from sklearn.metrics import roc_auc_score
from sklearn.ensemble import RandomForestClassifier

In [3]:
df_train = pd.read_csv("..\..\Project_DATA\HealthCareCat_Train.csv")
df_dev = pd.read_csv("..\..\Project_DATA\HealthCareCat_Dev.csv")
df_test = pd.read_csv("..\..\Project_DATA\HealthCareCat_test.csv")

In [5]:
allFeatures = pd.read_csv("..\..\Project_DATA\FeatureSelection_results.csv")
selectedRFFeatures = allFeatures[allFeatures.RF == 1]

featuresToDrop = np.setdiff1d(allFeatures.Feature,selectedRFFeatures.Feature)

df_train = df_train.drop(featuresToDrop, axis=1)
df_dev = df_dev.drop(featuresToDrop, axis=1)
df_test = df_test.drop(featuresToDrop, axis=1)


In [6]:
### 
X_train = df_train.drop("Spending_Cat",1)   #Feature Matrix
y_train = df_train["Spending_Cat"]          #Target Variable

X_dev = df_dev.drop("Spending_Cat",1)   #Feature Matrix
y_dev = df_dev["Spending_Cat"]          #Target Variable

X_test = df_test.drop("Spending_Cat",1)   #Feature Matrix
y_test = df_test["Spending_Cat"]          #Target Variable

In [22]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification

rf = RandomForestClassifier(random_state=0)
rf.fit(X_train,y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=0, verbose=0, warm_start=False)

In [83]:
results = pd.DataFrame(columns=['ID', 'Accuracy_1', 'Accuracy_2','Accuracy_3','Accuracy_4'])

In [24]:
yhat_train = rf.predict(X_train)
pd.crosstab(y_train,yhat_train, rownames=['Actual'], colnames=['Predicted'])

Predicted,1,2,3,4
Actual,,,,
1,80209,73,3,0
2,61,79923,0,0
3,3,2,80211,0
4,0,0,0,80263


In [84]:
yhat_dev = rf.predict(X_dev)
cross = pd.crosstab(y_dev,yhat_dev, rownames=['Actual'], colnames=['Predicted'])
cross

Predicted,1,2,3,4
Actual,,,,
1,18828,1045,75,1
2,658,19562,30,0
3,26,19,19973,0
4,0,0,0,19971


In [60]:
yhat_test = rf.predict(X_test)
pd.crosstab(y_test,yhat_test, rownames=['Actual'], colnames=['Predicted'])


Predicted,1,2,3,4
Actual,,,,
1,24528,473,46,0
2,137,790,9,0
3,24,20,190,1
4,3,2,1,31


In [85]:
results = results.append(
    {'ID': 0, 
     'Accuracy_1': cross[1][1]/cross.sum(axis=1)[1],
     'Accuracy_2': cross[2][2]/cross.sum(axis=1)[2],
     'Accuracy_3': cross[3][3]/cross.sum(axis=1)[3],
     'Accuracy_4': cross[4][4]/cross.sum(axis=1)[4]
    },ignore_index=True)
results

,ID,Accuracy_1,Accuracy_2,Accuracy_3,Accuracy_4
0,0.0,0.943807,0.966025,0.997752,1.0


#### The accuracy values are high, but because of dataset is imbalanced - it's doesn't important. The mostly important thing is how accurate are classified the 4 and 3 categories (mostly expensive groups of insured)

#### Random Hyperparameter Grid

In [9]:
from sklearn.model_selection import RandomizedSearchCV
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 1200, num = 5)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 6)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
print(random_grid)

{'n_estimators': [200, 450, 700, 950, 1200], 'max_features': ['auto', 'sqrt'], 'max_depth': [10, 30, 50, 70, 90, 110, None], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 4], 'bootstrap': [True, False]}


In [ ]:

rf1 = RandomForestClassifier()
rf_random = RandomizedSearchCV(estimator = rf1, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=3, random_state=42, n_jobs = -1)
# Fit the random search model
rf_random.fit(X_train, y_train)
rf_random.best_params_

Fitting 3 folds for each of 100 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed: 89.0min


#### After two unsuccesful attempts (every took more than 40 hours and crashed my laptop) I will continue with manual fine tuning

### 1.   Try n_estimators=200

In [7]:
rf1 = RandomForestClassifier(n_estimators=200)
rf1.fit(X_train,y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=200, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [86]:
yhat_dev = rf1.predict(X_dev)
cross = pd.crosstab(y_dev,yhat_dev, rownames=['Actual'], colnames=['Predicted'])
cross

### A bit better then previous model

Predicted,1,2,3,4
Actual,,,,
1,19139,809,1,0
2,272,19978,0,0
3,4,0,20014,0
4,0,0,0,19971


In [87]:
results = results.append(
    {'ID': 1, 
     'Accuracy_1': cross[1][1]/cross.sum(axis=1)[1],
     'Accuracy_2': cross[2][2]/cross.sum(axis=1)[2],
     'Accuracy_3': cross[3][3]/cross.sum(axis=1)[3],
     'Accuracy_4': cross[4][4]/cross.sum(axis=1)[4]
    },ignore_index=True)
results

,ID,Accuracy_1,Accuracy_2,Accuracy_3,Accuracy_4
0,0.0,0.943807,0.966025,0.997752,1.0
1,1.0,0.959396,0.986568,0.999800,1.0


### 2.  Try n_estimators=500

In [10]:
rf2 = RandomForestClassifier(n_estimators=500)
rf2.fit(X_train,y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=500, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [88]:
yhat_dev = rf2.predict(X_dev)
cross = pd.crosstab(y_dev,yhat_dev, rownames=['Actual'], colnames=['Predicted'])
cross

Predicted,1,2,3,4
Actual,,,,
1,19119,830,0,0
2,267,19983,0,0
3,4,2,20012,0
4,0,0,0,19971


In [89]:
results = results.append(
    {'ID': 2, 
     'Accuracy_1': cross[1][1]/cross.sum(axis=1)[1],
     'Accuracy_2': cross[2][2]/cross.sum(axis=1)[2],
     'Accuracy_3': cross[3][3]/cross.sum(axis=1)[3],
     'Accuracy_4': cross[4][4]/cross.sum(axis=1)[4]
    },ignore_index=True)
results

,ID,Accuracy_1,Accuracy_2,Accuracy_3,Accuracy_4
0,0.0,0.943807,0.966025,0.997752,1.0
1,1.0,0.959396,0.986568,0.999800,1.0
2,2.0,0.958394,0.986815,0.999700,1.0


### 3.  Stay with n_estimators=200, try max_features=sqrt

In [15]:
rf3 = RandomForestClassifier(n_estimators=200, max_features="sqrt")
rf3.fit(X_train,y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='sqrt', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=200, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [90]:
yhat_dev = rf3.predict(X_dev)
cross = pd.crosstab(y_dev,yhat_dev, rownames=['Actual'], colnames=['Predicted'])
cross

Predicted,1,2,3,4
Actual,,,,
1,19108,839,2,0
2,288,19962,0,0
3,4,1,20013,0
4,0,0,0,19971


In [91]:
results = results.append(
    {'ID': 3, 
     'Accuracy_1': cross[1][1]/cross.sum(axis=1)[1],
     'Accuracy_2': cross[2][2]/cross.sum(axis=1)[2],
     'Accuracy_3': cross[3][3]/cross.sum(axis=1)[3],
     'Accuracy_4': cross[4][4]/cross.sum(axis=1)[4]
    },ignore_index=True)
results

,ID,Accuracy_1,Accuracy_2,Accuracy_3,Accuracy_4
0,0.0,0.943807,0.966025,0.997752,1.0
1,1.0,0.959396,0.986568,0.999800,1.0
2,2.0,0.958394,0.986815,0.999700,1.0
3,3.0,0.957842,0.985778,0.999750,1.0


### 4.  Try max_depth = 10

In [17]:
rf4 = RandomForestClassifier(n_estimators=200, max_features="auto", max_depth = 10)
rf4.fit(X_train,y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=10, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=200, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [92]:
yhat_dev = rf4.predict(X_dev)
cross = pd.crosstab(y_dev,yhat_dev, rownames=['Actual'], colnames=['Predicted'])
cross

Predicted,1,2,3,4
Actual,,,,
1,17186,2389,306,68
2,5169,13434,1448,199
3,3092,1129,15751,46
4,298,20,17,19636


In [93]:
results = results.append(
    {'ID': 4, 
     'Accuracy_1': cross[1][1]/cross.sum(axis=1)[1],
     'Accuracy_2': cross[2][2]/cross.sum(axis=1)[2],
     'Accuracy_3': cross[3][3]/cross.sum(axis=1)[3],
     'Accuracy_4': cross[4][4]/cross.sum(axis=1)[4]
    },ignore_index=True)
results

,ID,Accuracy_1,Accuracy_2,Accuracy_3,Accuracy_4
0,0.0,0.943807,0.966025,0.997752,1.000000
1,1.0,0.959396,0.986568,0.999800,1.000000
2,2.0,0.958394,0.986815,0.999700,1.000000
3,3.0,0.957842,0.985778,0.999750,1.000000
4,4.0,0.861497,0.663407,0.786842,0.983226


### 5.  Try max_depth = 50

In [19]:
rf5 = RandomForestClassifier(n_estimators=200, max_features="auto", max_depth = 50)
rf5.fit(X_train,y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=50, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=200, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [94]:
yhat_dev = rf5.predict(X_dev)
cross = pd.crosstab(y_dev,yhat_dev, rownames=['Actual'], colnames=['Predicted'])
cross

Predicted,1,2,3,4
Actual,,,,
1,19133,815,1,0
2,291,19959,0,0
3,4,2,20012,0
4,0,0,0,19971


In [95]:
results = results.append(
    {'ID': 5, 
     'Accuracy_1': cross[1][1]/cross.sum(axis=1)[1],
     'Accuracy_2': cross[2][2]/cross.sum(axis=1)[2],
     'Accuracy_3': cross[3][3]/cross.sum(axis=1)[3],
     'Accuracy_4': cross[4][4]/cross.sum(axis=1)[4]
    },ignore_index=True)
results

,ID,Accuracy_1,Accuracy_2,Accuracy_3,Accuracy_4
0,0.0,0.943807,0.966025,0.997752,1.000000
1,1.0,0.959396,0.986568,0.999800,1.000000
2,2.0,0.958394,0.986815,0.999700,1.000000
3,3.0,0.957842,0.985778,0.999750,1.000000
4,4.0,0.861497,0.663407,0.786842,0.983226
5,5.0,0.959096,0.985630,0.999700,1.000000


### 6.  Try max_depth = 100

In [75]:
rf6 = RandomForestClassifier(n_estimators=200, max_features="auto", max_depth = 100)
rf6.fit(X_train,y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=100, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=200, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [96]:
yhat_dev = rf6.predict(X_dev)
cross = pd.crosstab(y_dev,yhat_dev, rownames=['Actual'], colnames=['Predicted'])
cross

Predicted,1,2,3,4
Actual,,,,
1,19125,822,2,0
2,266,19984,0,0
3,4,3,20011,0
4,0,0,0,19971


In [97]:
results = results.append(
    {'ID': 6, 
     'Accuracy_1': cross[1][1]/cross.sum(axis=1)[1],
     'Accuracy_2': cross[2][2]/cross.sum(axis=1)[2],
     'Accuracy_3': cross[3][3]/cross.sum(axis=1)[3],
     'Accuracy_4': cross[4][4]/cross.sum(axis=1)[4]
    },ignore_index=True)
results

,ID,Accuracy_1,Accuracy_2,Accuracy_3,Accuracy_4
0,0.0,0.943807,0.966025,0.997752,1.000000
1,1.0,0.959396,0.986568,0.999800,1.000000
2,2.0,0.958394,0.986815,0.999700,1.000000
3,3.0,0.957842,0.985778,0.999750,1.000000
4,4.0,0.861497,0.663407,0.786842,0.983226
5,5.0,0.959096,0.985630,0.999700,1.000000
6,6.0,0.958695,0.986864,0.999650,1.000000


### 7.  Try max_depth = None

In [79]:
rf7 = RandomForestClassifier(n_estimators=200, max_features="auto", max_depth=None)
rf7.fit(X_train,y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=200, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [98]:
yhat_dev = rf7.predict(X_dev)
cross = pd.crosstab(y_dev,yhat_dev, rownames=['Actual'], colnames=['Predicted'])
cross

Predicted,1,2,3,4
Actual,,,,
1,19130,819,0,0
2,286,19964,0,0
3,5,2,20011,0
4,0,0,0,19971


In [99]:
results = results.append(
    {'ID': 7, 
     'Accuracy_1': cross[1][1]/cross.sum(axis=1)[1],
     'Accuracy_2': cross[2][2]/cross.sum(axis=1)[2],
     'Accuracy_3': cross[3][3]/cross.sum(axis=1)[3],
     'Accuracy_4': cross[4][4]/cross.sum(axis=1)[4]
    },ignore_index=True)
results

,ID,Accuracy_1,Accuracy_2,Accuracy_3,Accuracy_4
0,0.0,0.943807,0.966025,0.997752,1.000000
1,1.0,0.959396,0.986568,0.999800,1.000000
2,2.0,0.958394,0.986815,0.999700,1.000000
3,3.0,0.957842,0.985778,0.999750,1.000000
4,4.0,0.861497,0.663407,0.786842,0.983226
5,5.0,0.959096,0.985630,0.999700,1.000000
6,6.0,0.958695,0.986864,0.999650,1.000000
7,7.0,0.958945,0.985877,0.999650,1.000000


### Find the best parameters for groups 4 and 3. And try it on test dataframe:

In [100]:
results["Sum"] = results.Accuracy_4 + results.Accuracy_3
results

,ID,Accuracy_1,Accuracy_2,Accuracy_3,Accuracy_4,Sum
0,0.0,0.943807,0.966025,0.997752,1.000000,1.997752
1,1.0,0.959396,0.986568,0.999800,1.000000,1.999800
2,2.0,0.958394,0.986815,0.999700,1.000000,1.999700
3,3.0,0.957842,0.985778,0.999750,1.000000,1.999750
4,4.0,0.861497,0.663407,0.786842,0.983226,1.770068
5,5.0,0.959096,0.985630,0.999700,1.000000,1.999700
6,6.0,0.958695,0.986864,0.999650,1.000000,1.999650
7,7.0,0.958945,0.985877,0.999650,1.000000,1.999650


In [101]:
### Its number 1:

yhat_test = rf1.predict(X_test)
cross = pd.crosstab(y_test,yhat_test, rownames=['Actual'], colnames=['Predicted'])
cross


Predicted,1,2,3,4
Actual,,,,
1,24686,361,0,0
2,121,815,0,0
3,20,25,190,0
4,1,5,1,30


In [104]:
res = {
     'Accuracy_1': cross[1][1]/cross.sum(axis=1)[1],
     'Accuracy_2': cross[2][2]/cross.sum(axis=1)[2],
     'Accuracy_3': cross[3][3]/cross.sum(axis=1)[3],
     'Accuracy_4': cross[4][4]/cross.sum(axis=1)[4],
     'Accuracy_Total': (cross[1][1] + cross[2][2] + cross[3][3] + cross[4][4]) / (cross.sum(axis=1)[1] + cross.sum(axis=1)[2] + cross.sum(axis=1)[3] + cross.sum(axis=1)[4])
    }
res

{'Accuracy_1': 0.9855870962590331,
 'Accuracy_2': 0.8707264957264957,
 'Accuracy_3': 0.8085106382978723,
 'Accuracy_4': 0.8108108108108109,
 'Accuracy_Total': 0.9796610169491525}